# LAB.2
## Homework 2: Discovery of Frequent Itemsets and Association Rules

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import os
import sys
sys.path.append('/content/drive/MyDrive/DM_Labs/dataset')

import matplotlib.pyplot as plt
from itertools import chain, combinations
import statistics as stat
import pandas as pd
import numpy as np
import csv
import re

#Pre-processing

In [11]:
raw_df = pd.read_csv("/content/drive/MyDrive/DM_Labs/dataset/T10I4D100K.dat", header=None, sep='\s\s+', engine='python')

$data$ is the list of baskets.

$items$ is the set $I$ of items.

In [12]:
baskets = []
items = set()

for i in range(len(raw_df[0])):
  basket = [int(s) for s in re.findall(r'\b\d+\b', raw_df[0][i])]
  baskets.append(basket)
  items.update(basket)

In [13]:
len(items)

870

# Support

**supportify** creates a dictionary <item, occurrences> with all the occurrences $\ge$ support.

In [8]:
def supportify(baskets, support=1):
  support_dict = {}

  for basket in baskets:
    for item in basket:
      
      if item not in support_dict:
        support_dict[item] = 1
        continue

      support_dict[item] = support_dict[item] + 1
  
  return {k: v for k, v in support_dict.items() if v >= support}

In [14]:
support_dict = supportify(baskets)

In [15]:
stat.median(support_dict.values())

816.5

Setting the support at 817

In [16]:
support = 817

In [17]:
support_dict = supportify(baskets, support)

In [18]:
len(support_dict)

435

In the previous example, we can see that half of the items were cut off.

#A-priori

**candidate_constructor** takes a set of itemsets and a set of infrequents and construct all the candidates of size *len(itemset)+1*. For example, taken { (a,b), (b,d) } it constructs { (a,b,d) }. The infrequent set is passed to make sure that at step $k$ we don't construct candidates that contain infrequent sets at steps $1, ... , k-1$.

In [19]:
def candidate_constructor(itemsets, infrequents=()):
  result = set()
  itemsets = list(itemsets)
  size = len(itemsets[0])
  
  if len(itemsets) == 1:
    return set(itemsets)

  for i in range(len(itemsets)-1):
    for j in range(i+1, len(itemsets)):
      for el in itemsets[j]:
        temp = frozenset(itemsets[i]).union(frozenset([el]))
        if len(temp) == size + 1:
          result.add(temp)
  
  if len(infrequents) == 0:
    return result
  else:
    res2 = result.copy()
    for itemset in result: # Check if itemsets contain infrequent subsets
      for infrequent in infrequents:
        if infrequent & itemset == infrequent:
          if itemset in res2:
            res2.remove(itemset)
            
          
        
  return res2

**filter** takes candidates as input and a level of support and filters out all the candidates below the support level.

In [37]:
def filter(candidates, support=1):
  
  support_dict = {}
  infrequents = set()
  '''
  baskets = [
      ['b', 'm', 'c'],
      ['b', 'c', 'j'],
      ['a', 'b', 'c'],
      ['b', 'm'],
      ['b', 'c']
  ]
  #'''
  i=1
  for basket in baskets:
    
    if i % 5000 == 0:
      print(str(i/len(baskets) * 100) + "%")

    for candidate in candidates:
      if candidate & frozenset(basket) == candidate:
        
        if candidate not in support_dict:
          support_dict[candidate] = 1
          continue
        support_dict[candidate] = support_dict[candidate] + 1

    i += 1

  dic2 = support_dict.copy()
  for k, v in support_dict.items():
    if v < support:
      dic2.pop(k)
      infrequents.add(frozenset(k))

  return dic2, infrequents

Here is the process:

In [38]:
support = 3500

In [39]:
basketset = supportify(baskets, support)
temp = set()
for item in list(basketset.keys()):
  temp.add(frozenset([item]))
candidates = candidate_constructor(temp)

In [40]:
len(candidates) # n(n-1)/2 = 780

780

For more supported itemsets, lower the support.

In [41]:
%%time
supported, infrequent = filter(candidates, support=300)

5.0%
10.0%
15.0%
20.0%
25.0%
30.0%
35.0%
40.0%
45.0%
50.0%
55.00000000000001%
60.0%
65.0%
70.0%
75.0%
80.0%
85.0%
90.0%
95.0%
100.0%
CPU times: user 44.8 s, sys: 128 ms, total: 45 s
Wall time: 45.3 s


In [42]:
print(len(supported), len(infrequent))
support_list = []
support_list.append(supported)
supported

151 629


{frozenset({368, 538}): 479,
 frozenset({205, 401}): 313,
 frozenset({120, 205}): 373,
 frozenset({766, 883}): 348,
 frozenset({682, 883}): 400,
 frozenset({529, 883}): 449,
 frozenset({529, 947}): 422,
 frozenset({71, 529}): 304,
 frozenset({529, 720}): 442,
 frozenset({217, 529}): 403,
 frozenset({177, 775}): 510,
 frozenset({217, 888}): 539,
 frozenset({71, 766}): 673,
 frozenset({419, 722}): 366,
 frozenset({145, 283}): 368,
 frozenset({401, 684}): 467,
 frozenset({354, 775}): 305,
 frozenset({789, 829}): 1194,
 frozenset({529, 829}): 584,
 frozenset({684, 829}): 349,
 frozenset({460, 684}): 619,
 frozenset({39, 722}): 370,
 frozenset({914, 919}): 375,
 frozenset({368, 829}): 1194,
 frozenset({368, 914}): 312,
 frozenset({368, 419}): 355,
 frozenset({541, 937}): 458,
 frozenset({529, 541}): 395,
 frozenset({368, 529}): 640,
 frozenset({368, 947}): 442,
 frozenset({368, 937}): 701,
 frozenset({529, 937}): 491,
 frozenset({937, 947}): 367,
 frozenset({541, 947}): 365,
 frozenset({368

In [43]:
infrequent_set = infrequent

In [44]:
temp = set()
for item in list(supported.keys()):
  temp.add(item)
candidates = candidate_constructor(temp, set(infrequent))

In [45]:
print(len(candidates))
candidates

140


{frozenset({39, 354, 722}),
 frozenset({401, 460, 684}),
 frozenset({368, 494, 682}),
 frozenset({438, 529, 684}),
 frozenset({368, 684, 766}),
 frozenset({368, 470, 494}),
 frozenset({368, 529, 538}),
 frozenset({368, 766, 829}),
 frozenset({368, 529, 829}),
 frozenset({32, 494, 937}),
 frozenset({217, 529, 947}),
 frozenset({692, 789, 829}),
 frozenset({354, 722, 966}),
 frozenset({460, 529, 538}),
 frozenset({368, 529, 720}),
 frozenset({120, 368, 766}),
 frozenset({39, 145, 368}),
 frozenset({368, 460, 529}),
 frozenset({354, 529, 766}),
 frozenset({71, 354, 766}),
 frozenset({205, 368, 538}),
 frozenset({145, 494, 937}),
 frozenset({39, 368, 470}),
 frozenset({529, 766, 829}),
 frozenset({368, 692, 829}),
 frozenset({368, 529, 766}),
 frozenset({529, 937, 947}),
 frozenset({71, 438, 684}),
 frozenset({39, 419, 722}),
 frozenset({684, 766, 829}),
 frozenset({494, 682, 883}),
 frozenset({494, 541, 937}),
 frozenset({684, 722, 766}),
 frozenset({217, 419, 722}),
 frozenset({120, 438,

In [46]:
%%time
supported, infrequent = filter(candidates, support=300)

5.0%
10.0%
15.0%
20.0%
25.0%
30.0%
35.0%
40.0%
45.0%
50.0%
55.00000000000001%
60.0%
65.0%
70.0%
75.0%
80.0%
85.0%
90.0%
95.0%
100.0%
CPU times: user 11.8 s, sys: 40.7 ms, total: 11.9 s
Wall time: 12 s


These are the candidates filtering singletons with $s=3500$, doubletons and tripletons with $s=300$

In [47]:
len(supported)
support_list.append(supported)
print(supported)

{frozenset({368, 682, 494}): 591, frozenset({368, 470, 39}): 332, frozenset({145, 419, 39}): 352}


It seems like we found the most popular sets, we are going to stop here just for the sake of the exercise and move forward to the **Rule Association** creation.

# Association Rules

Already found all the doubletons and tripletons with support 300.

In [26]:
support = 300
confidence = 0.5

**find_confidence** calculates the confidence of a certain rule taking the arguments from_set -> to_set of the rule.

In [27]:
def find_confidence(from_set, to_set):
  from_set_count = 0
  total_set_count = 0
  
  total_set = from_set.union(to_set)

  for basket in baskets:
    setbasket = frozenset(basket)
    if from_set & setbasket == from_set:
      from_set_count += 1
    if total_set & setbasket == total_set:
      total_set_count += 1
  
  return total_set_count / from_set_count

**powerset** creates all the subsets of a given set, including the emptyset and the set itself.

In [28]:
def powerset(itemset):
    s = list(itemset)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

**mining_rules** takes a support list with all the doubletons, tripletons, etc.. and creates all the possible subsets $A$, evaluating the confidence of all the rules $A \implies I-A$. It keeps only the rules above a certain confidence given as input.

In [44]:
def mining_rules(support_list, confidence):
  rules = list()
  for sets in support_list:
    
    for itemset in sets:
      all_subsets = powerset(itemset)
      for subset in (subset for subset in all_subsets if len(subset) > 0 and len(subset) < len(itemset)): # all the subsets which are not the superset or empty set
        difference = set(itemset) - set(subset)
        #print(set(subset), difference)
        conf = find_confidence(set(subset), difference)
        #print(conf)
        if conf >= confidence:
          rules.append((str(subset) + '->' + str(difference), conf))

  return rules


In [45]:
%%time
rules = mining_rules(support_list, confidence=0.3)
rules

CPU times: user 28.6 s, sys: 83.2 ms, total: 28.7 s
Wall time: 28.7 s


[('(368, 682)->{494}', 0.4953897736797988),
 ('(368, 494)->{682}', 0.6872093023255814),
 ('(682, 494)->{368}', 0.8277310924369747),
 ('(368, 470)->{39}', 0.5714285714285714),
 ('(368, 39)->{470}', 0.5876106194690266),
 ('(470, 39)->{368}', 0.7094017094017094),
 ('(145, 419)->{39}', 0.5578446909667195),
 ('(145, 39)->{419}', 0.47956403269754766),
 ('(419, 39)->{145}', 0.6888454011741683)]

#Testing

Some tries:

In [48]:
candidate_constructor((frozenset('b'), frozenset('c'), frozenset('j'), frozenset('m')))

{frozenset({'b', 'c'}),
 frozenset({'b', 'j'}),
 frozenset({'b', 'm'}),
 frozenset({'j', 'm'}),
 frozenset({'c', 'j'}),
 frozenset({'c', 'm'})}

support = 1

In [49]:
print(candidate_constructor((frozenset(('f', 'b')), frozenset(('b', 'm')), frozenset(('c', 'b')), frozenset(('j', 'c')), frozenset(('m', 'c')))))
s, inf = filter(candidate_constructor((frozenset(('b', 'm')), frozenset(('c', 'b')), frozenset(('j', 'c')), frozenset(('m', 'c'))), infrequents=((frozenset('k'), frozenset(('j', 'm')))) ))
print(s)
print(inf)

{frozenset({'j', 'b', 'c'}), frozenset({'b', 'm', 'f'}), frozenset({'j', 'b', 'm'}), frozenset({'j', 'b', 'f'}), frozenset({'b', 'c', 'f'}), frozenset({'j', 'm', 'c'}), frozenset({'b', 'm', 'c'})}
5.0%
10.0%
15.0%
20.0%
25.0%
30.0%
35.0%
40.0%
45.0%
50.0%
55.00000000000001%
60.0%
65.0%
70.0%
75.0%
80.0%
85.0%
90.0%
95.0%
100.0%
{}
set()


support = 2

In [50]:
filter(candidate_constructor((frozenset(('b', 'm')), frozenset(('c', 'b')), frozenset(('j', 'c')), frozenset(('m', 'c'))), infrequents=((frozenset('k'), frozenset(('j', 'm')))) ), support=2)

5.0%
10.0%
15.0%
20.0%
25.0%
30.0%
35.0%
40.0%
45.0%
50.0%
55.00000000000001%
60.0%
65.0%
70.0%
75.0%
80.0%
85.0%
90.0%
95.0%
100.0%


({}, set())

In [51]:
i = 0
j = 0
for basket in baskets:
  if 538 in basket and 368 in basket:
    i += 1
  if 538 in basket:
    j += 1
print(i/j)

0.12029131089904571


In [52]:
i = 0
j = 0
for basket in baskets:
  if 682 in basket and 494 in basket and 368 in basket:
    i += 1
  if 682 in basket and 494 in basket:
    j += 1
print(i/j)

0.8277310924369747


In [53]:
i = 0
for basket in baskets:
  if 368 in basket and 682 in basket and 494 in basket:
    i += 1
print(i)

591


# To html

In [2]:
%%shell
jupyter nbconvert --to html /content/drive/MyDrive/DM_Labs/Lab2.ipynb

[NbConvertApp] Converting notebook /content/drive/MyDrive/DM_Labs/Lab2.ipynb to html
[NbConvertApp] Writing 337044 bytes to /content/drive/MyDrive/DM_Labs/Lab2.html
